## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Extract Adverse Drug Events (ADE)](https://aws.amazon.com/marketplace/pp/prodview-sueq4zd6lhe7e)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Background 


- **Model**: `en.explain_doc.clinical_ade`

- **Model Description**: This pipeline will classify the document, extract ADE and DRUG clinical entities, assign assertion status to ADE entities, and relate Drugs with their ADEs.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "jsl-explain-doc-clinical-ade"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None) 


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)    
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'
        
        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
        
        with open(input_file_name, 'w') as f:
            f.write(json_input_data)
        
        s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-input-json/real-time/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))
        
        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))
        df_assertion = pd.DataFrame(response_data['assertion_predictions'])
        df_relation = pd.DataFrame(response_data['relation_predictions'])
    
        display(df_assertion, df_relation)
       

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


  **Input format**:
  
  
  {"**text**": "Input Text from which biology and genetics terms are to be extracted."}

In [7]:
data_dicts = [ {"text" :["Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",
    "Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain."
  ]}
             ]

process_data_and_invoke_realtime_endpoint(data_dicts)


,document,ner_chunk,entity,begin,end,confidence,assertion
0,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",Lipitor,DRUG,12,18,0.9998,-
1,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",severe fatigue,ADE,51,64,0.78,conditional
2,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",voltaren,DRUG,95,102,0.9992,-
3,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",cramps,ADE,148,153,0.988,present
4,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Zocor,DRUG,8,12,0.9997,-
5,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",constant headaches,ADE,42,59,0.7586,present
6,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",dizziness,ADE,65,73,0.9985,present
7,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Celebrex,DRUG,104,111,0.9988,-
8,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",occasional joint pain,ADE,164,184,0.8241,present


,document,ner_chunk_1,entity_1,ner_chunk_1_begin,ner_chunk_1_end,ner_chunk_2,entity_2,ner_chunk_2_begin,ner_chunk_2_end,relation,confidence
0,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",Lipitor,DRUG,12,18,severe fatigue,ADE,51,64,1,1.0
1,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",Lipitor,DRUG,12,18,cramps,ADE,148,153,1,1.0
2,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",severe fatigue,ADE,51,64,voltaren,DRUG,95,102,1,0.9661623
3,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",voltaren,DRUG,95,102,cramps,ADE,148,153,1,0.99999547
4,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Zocor,DRUG,8,12,constant headaches,ADE,42,59,1,0.99429744
5,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Zocor,DRUG,8,12,dizziness,ADE,65,73,1,0.998899
6,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Zocor,DRUG,8,12,occasional joint pain,ADE,164,184,1,0.9984377
7,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",constant headaches,ADE,42,59,Celebrex,DRUG,104,111,0,0.99999964
8,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",dizziness,ADE,65,73,Celebrex,DRUG,104,111,0,1.0
9,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Celebrex,DRUG,104,111,occasional joint pain,ADE,164,184,1,0.99807066


In [8]:
data_dicts = [
    {"text" :  "Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness."}
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,entity,begin,end,confidence,assertion
0,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",Crestor,DRUG,5,11,0.9983,-
1,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",muscle aches,ADE,41,52,0.7823,present
2,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",fatigue,ADE,58,64,0.9402,present
3,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",Naproxen,DRUG,97,104,0.9903,-
4,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",muscle soreness,ADE,138,152,0.76125,present


,document,ner_chunk_1,entity_1,ner_chunk_1_begin,ner_chunk_1_end,ner_chunk_2,entity_2,ner_chunk_2_begin,ner_chunk_2_end,relation,confidence
0,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",Crestor,DRUG,5,11,muscle aches,ADE,41,52,1,0.99999917
1,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",Crestor,DRUG,5,11,fatigue,ADE,58,64,1,0.99999964
2,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",Crestor,DRUG,5,11,muscle soreness,ADE,138,152,1,0.9956077
3,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",muscle aches,ADE,41,52,Naproxen,DRUG,97,104,0,1.0
4,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",fatigue,ADE,58,64,Naproxen,DRUG,97,104,0,1.0
5,"Used Crestor for 10+ years with frequent muscle aches and fatigue. Doctor recently changed me to Naproxen, now experiencing less frequent muscle soreness.",Naproxen,DRUG,97,104,muscle soreness,ADE,138,152,1,0.99999976


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [9]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [10]:
validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input-json/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output-json/batch"

In [11]:
input_file_name = 'inputs/batch/input.json'
output_file_name = 'outputs/batch/out.out'

os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

In [12]:
validation_file_name = "input.json"
input_json_data = {"text" :["Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",
    "Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain."
  ]}
json_input_data = json.dumps(input_json_data)
with open(input_file_name, 'w') as f:
    f.write(json_input_data)

s3_client.put_object(
    Bucket=s3_bucket,
    Key=f"{model_name}/validation-input-json/batch/{validation_file_name}",
    Body=(bytes(json_input_data.encode("UTF-8"))),
)

{'ResponseMetadata': {'RequestId': 'YDQ3HWPMJT6J5FZJ',
  'HostId': '01DfGE6ma3f6Q2oDd/VuGMiFu2jWbYJzsJzShztsCpQxT90nJMwwMKz7pTynikC1lxApMSDsEmT9J1fp+7PYYw==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '01DfGE6ma3f6Q2oDd/VuGMiFu2jWbYJzsJzShztsCpQxT90nJMwwMKz7pTynikC1lxApMSDsEmT9J1fp+7PYYw==',
   'x-amz-request-id': 'YDQ3HWPMJT6J5FZJ',
   'date': 'Mon, 11 Mar 2024 14:33:58 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"caf4eb897d2f423ce2902a2e0e47a05a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"caf4eb897d2f423ce2902a2e0e47a05a"',
 'ServerSideEncryption': 'AES256'}

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [14]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

data = json.loads(response["Body"].read().decode("utf-8"))
df_assertion = pd.DataFrame(data['assertion_predictions'])
df_relation = pd.DataFrame(data['relation_predictions'])
display(df_assertion, df_relation)

with open(output_file_name, 'w') as f_out:
    json.dump(data, f_out, indent=4)

,document,ner_chunk,entity,begin,end,confidence,assertion
0,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",Lipitor,DRUG,12,18,0.9998,-
1,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",severe fatigue,ADE,51,64,0.78,conditional
2,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",voltaren,DRUG,95,102,0.9992,-
3,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",cramps,ADE,148,153,0.988,present
4,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Zocor,DRUG,8,12,0.9997,-
5,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",constant headaches,ADE,42,59,0.7586,present
6,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",dizziness,ADE,65,73,0.9985,present
7,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Celebrex,DRUG,104,111,0.9988,-
8,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",occasional joint pain,ADE,164,184,0.8241,present


,document,ner_chunk_1,entity_1,ner_chunk_1_begin,ner_chunk_1_end,ner_chunk_2,entity_2,ner_chunk_2_begin,ner_chunk_2_end,relation,confidence
0,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",Lipitor,DRUG,12,18,severe fatigue,ADE,51,64,1,1.0
1,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",Lipitor,DRUG,12,18,cramps,ADE,148,153,1,1.0
2,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",severe fatigue,ADE,51,64,voltaren,DRUG,95,102,1,0.96616125
3,"Been taking Lipitor for 15 years, have experienced severe fatigue a lot!!!. Doctor moved me to voltaren 2 months ago, so far, have only experienced cramps",voltaren,DRUG,95,102,cramps,ADE,148,153,1,0.99999547
4,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Zocor,DRUG,8,12,constant headaches,ADE,42,59,1,0.9942973
5,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Zocor,DRUG,8,12,dizziness,ADE,65,73,1,0.998899
6,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Zocor,DRUG,8,12,occasional joint pain,ADE,164,184,1,0.9984377
7,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",constant headaches,ADE,42,59,Celebrex,DRUG,104,111,0,0.99999964
8,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",dizziness,ADE,65,73,Celebrex,DRUG,104,111,0,1.0
9,"Been on Zocor for over a decade, enduring constant headaches and dizziness. My physician switched me to Celebrex recently, and since then, I have mostly dealt with occasional joint pain.",Celebrex,DRUG,104,111,occasional joint pain,ADE,164,184,1,0.99807066


In [15]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-explain-doc-clinical-ade-2024-03-11-14-33-57-169


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

